# Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

# Importing Data


In [2]:
# Creating a Path
path = r'/Users/jacob/Desktop/Instacart Basket Analysis'

In [3]:
high_activity_customers = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'high_activity_customers.pkl'))

## 5. Creating a profiling variable based on age, income, certain goods in the “department_id” column, and number of dependents. We will use the “orders_day_of_the_week” and “order_hour_of_day” columns because they might impact customer profiles.

## Age Groups

In [5]:
# Determining age groups and creating flag

high_activity_customers['age'].min()

18

In [6]:
# Determining age groups and creating flag

high_activity_customers['age'].max()

81

In [7]:
# Creating a flag 'age_group'
high_activity_customers.loc[high_activity_customers['age'] <= 29, 'age_group'] = '18-29'
high_activity_customers.loc[(high_activity_customers['age'] > 29) & (high_activity_customers['age'] < 40), 'age_group'] = '30-39'
high_activity_customers.loc[(high_activity_customers['age'] > 39) & (high_activity_customers['age'] < 50), 'age_group'] = '40-49'
high_activity_customers.loc[(high_activity_customers['age'] > 49) & (high_activity_customers['age'] < 60), 'age_group'] = '50-59'
high_activity_customers.loc[(high_activity_customers['age'] > 59) & (high_activity_customers['age'] < 70), 'age_group'] = '60-69'
high_activity_customers.loc[high_activity_customers['age'] > 69, 'age_group'] = '70 and over'

In [8]:
# Checking that total of column values equals total rows in pickle

high_activity_customers['age_group'].value_counts(dropna = False)

age_group
18-29          5817603
70 and over    5812883
40-49          4883161
30-39          4847506
50-59          4841771
60-69          4761621
Name: count, dtype: int64

## Income Group

In [9]:
# Determining income groups and creating flag

high_activity_customers['income'].min()

25903

In [10]:
# Determining income groups and creating flag

high_activity_customers['income'].max()

593901

In [11]:
# Creating a flag 'income_group' 

high_activity_customers.loc[high_activity_customers['income'] <= 75000, 'income_group'] = 'Working Class'
high_activity_customers.loc[(high_activity_customers['income'] > 75000) & (high_activity_customers['income'] < 120001), 'income_group'] = 'Middle Class'
high_activity_customers.loc[(high_activity_customers['income'] > 120000) & (high_activity_customers['income'] < 400001), 'income_group'] = 'Upper Class'
high_activity_customers.loc[high_activity_customers['income'] > 400000, 'income_group'] = 'High Wealth Class'

In [12]:
# Checking that total of column values equals total rows in pickle

high_activity_customers['income_group'].value_counts(dropna = False)

income_group
Middle Class         11878317
Working Class         9906733
Upper Class           9132499
High Wealth Class       46996
Name: count, dtype: int64

## Departments

In [13]:
# Importing departments data set

df_departments = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'departments_wrangled.csv'), index_col = False)

In [14]:
# Rename column
df_departments = df_departments.rename(columns={'Unnamed: 0': 'department_id'})

In [16]:
# Print output
df_departments 

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol
5,6,international
6,7,beverages
7,8,pets
8,9,dry goods pasta
9,10,bulk


In [17]:
# Merging the departments dataframe with the high_activity_customers dataframe

high_activity_customers = high_activity_customers.merge(df_departments, on = 'department_id')

In [25]:
# Print output
high_activity_customers.columns

Index(['user_id', 'gender', 'state', 'age', 'date_joined',
       'number_of_dependants', 'fam_status', 'income', 'order_id',
       'order_number', 'orders_day_of_week', 'order_hour_of_day',
       'days_since_prior_order', 'product_id', 'add_to_cart_order',
       'reordered', 'product_name', 'aisle_id', 'department_id', 'prices',
       '_merge', 'price_range_loc', 'busiest_day', 'busiest_days',
       'busiest_period_of_day', 'max_order', 'loyalty_flag', 'average_price',
       'spending_flag', 'median_prior_orders', 'order_frequency_flag',
       'region', 'customer_activity', 'age_group', 'income_group',
       'department', 'family_type'],
      dtype='object')

In [27]:
# Creating family_type group

high_activity_customers.loc[(high_activity_customers['fam_status'].isin(['divorced/widowed', 'living with parents and siblings', 'single'])) & (high_activity_customers['number_of_dependants'] == 0), 'family_type'] = 'single no dependants'
high_activity_customers.loc[(high_activity_customers['fam_status'].isin(['divorced/widowed', 'living with parents and siblings', 'single'])) & (high_activity_customers['number_of_dependants'] > 0), 'family_type'] = 'Single with dependants'
high_activity_customers.loc[(high_activity_customers['fam_status'] == 'married') & (high_activity_customers['number_of_dependants'] == 1), 'family_type'] = 'married no dependants'
high_activity_customers.loc[(high_activity_customers['fam_status'] == 'married') & (high_activity_customers['number_of_dependants'] > 1), 'family_type'] = 'married with dependants'

In [29]:
# Print Output
high_activity_customers['family_type'].value_counts()

family_type
married with dependants    14532632
single no dependants        7739680
married no dependants       7211061
Single with dependants      1481172
Name: count, dtype: int64

In [32]:
# Counting values for days of the week where customers shop the most, being 0 = Saturday and so on
high_activity_customers['orders_day_of_week'].value_counts(dropna = False)

orders_day_of_week
0    5908610
1    5411676
6    4291303
5    4033726
2    4024704
3    3669532
4    3624994
Name: count, dtype: int64

In [42]:
# Creating buyer_type group
high_activity_customers.loc[(high_activity_customers['orders_day_of_week'].isin([0, 1])) , 'buyer_type'] = 'Weekend Buyer'
high_activity_customers.loc[(high_activity_customers['orders_day_of_week'].isin([2, 3, 4, 5, 6])) , 'buyer_type'] = 'Weekly Buyer'

In [43]:
high_activity_customers['buyer_type'].value_counts(dropna = False)

buyer_type
Weekly Buyer     19644259
Weekend Buyer    11320286
Name: count, dtype: int64

In [52]:
# Counting values for hours of the day where customers shop the most
high_activity_customers['order_hour_of_day'].value_counts(dropna = False)

order_hour_of_day
10    2649744
11    2614876
14    2567903
13    2539276
15    2535960
12    2495795
16    2413681
9     2363606
17    1982343
8     1659108
18    1551184
19    1193878
20     930909
7      863213
21     763111
22     607060
23     384554
6      281013
0      208042
1      109621
5       84418
2       65691
4       50763
3       48796
Name: count, dtype: int64

In [53]:
# Creating buyer_type_hour group
high_activity_customers.loc[high_activity_customers['order_hour_of_day'] >= 12, 'buyer_type_hour'] = 'Late Buyer'
high_activity_customers.loc[high_activity_customers['order_hour_of_day'] < 12, 'buyer_type_hour'] = 'Early Buyer'


In [51]:
high_activity_customers['buyer_type_hour'].value_counts(dropna = False)

buyer_type_hour
Late Buyer     19965654
Early Buyer    10998891
Name: count, dtype: int64

In [54]:
high_activity_customers.head()

,user_id,gender,state,age,date_joined,number_of_dependants,fam_status,income,order_id,order_number,...,order_frequency_flag,region,customer_activity,age_group,income_group,department,family_type,buyer_type,buyer_type_hour,buyer_type_hour1
0,33890,Female,New Mexico,36,1/1/2017,0,single,59285,474720,4,...,Non-frequent customer,West,high activity,30-39,Working Class,beverages,single no dependants,Weekend Buyer,Late Buyer,Late Buyer
1,33890,Female,New Mexico,36,1/1/2017,0,single,59285,896617,5,...,Non-frequent customer,West,high activity,30-39,Working Class,beverages,single no dependants,Weekly Buyer,Late Buyer,Late Buyer
2,33890,Female,New Mexico,36,1/1/2017,0,single,59285,1044841,3,...,Non-frequent customer,West,high activity,30-39,Working Class,beverages,single no dependants,Weekly Buyer,Early Buyer,Early Buyer
3,65803,Male,Idaho,35,1/1/2017,2,married,99568,3233494,6,...,Frequent customer,West,high activity,30-39,Middle Class,beverages,married with dependants,Weekly Buyer,Late Buyer,Late Buyer
4,65803,Male,Idaho,35,1/1/2017,2,married,99568,142275,8,...,Frequent customer,West,high activity,30-39,Middle Class,beverages,married with dependants,Weekend Buyer,Late Buyer,Late Buyer


In [55]:
high_activity_customers = high_activity_customers.drop('buyer_type_hour1', axis=1)


In [56]:
high_activity_customers.head()

,user_id,gender,state,age,date_joined,number_of_dependants,fam_status,income,order_id,order_number,...,median_prior_orders,order_frequency_flag,region,customer_activity,age_group,income_group,department,family_type,buyer_type,buyer_type_hour
0,33890,Female,New Mexico,36,1/1/2017,0,single,59285,474720,4,...,26.0,Non-frequent customer,West,high activity,30-39,Working Class,beverages,single no dependants,Weekend Buyer,Late Buyer
1,33890,Female,New Mexico,36,1/1/2017,0,single,59285,896617,5,...,26.0,Non-frequent customer,West,high activity,30-39,Working Class,beverages,single no dependants,Weekly Buyer,Late Buyer
2,33890,Female,New Mexico,36,1/1/2017,0,single,59285,1044841,3,...,26.0,Non-frequent customer,West,high activity,30-39,Working Class,beverages,single no dependants,Weekly Buyer,Early Buyer
3,65803,Male,Idaho,35,1/1/2017,2,married,99568,3233494,6,...,7.0,Frequent customer,West,high activity,30-39,Middle Class,beverages,married with dependants,Weekly Buyer,Late Buyer
4,65803,Male,Idaho,35,1/1/2017,2,married,99568,142275,8,...,7.0,Frequent customer,West,high activity,30-39,Middle Class,beverages,married with dependants,Weekend Buyer,Late Buyer


# Exporting and saving new dataframe high_activity_customers

In [57]:
high_activity_customers.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_custs_depts.pkl'))